<a href="https://colab.research.google.com/github/SKrishna93/EVA5/blob/master/EVA5_Session_4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Importing torch packages for Neural Network building
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [22]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__() 
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1), # Input=28x28x1 Kernel=3x3x1x8 Output=28x28x8 RF=3x3
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.Conv2d(8, 10, 3, padding=1), # Input=28x28x8 Kernel=3x3x8x10 Output=28x28x10 RF=5x5
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Conv2d(10, 10, 3, padding=1), #Input=28x28x10 Kernel=3x3x10x10 Output=28x28x10 RF=7x7
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.MaxPool2d(2, 2), # Input=28x28x10 Output=14x14x10 RF=14x14
        nn.Dropout(0.1)
        ) #First Convolution and Transistion Block. RF achieved 7x7 during Convolution.

    self.conv2 = nn.Sequential(
        nn.Conv2d(10, 10, 3, padding=1), #Input=14x14x10 Kernel=3x3x10x10 Output=14x14x10 RF=16x16
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Conv2d(10, 16, 3, padding=1), #Input=14x14x10 Kernel=3x3x10x16 Output=14x14x16 RF=18x18
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.MaxPool2d(2, 2), #Input=14x14x16 Output=7x7x16 RF=20x20
        nn.Dropout(0.05)
        ) #Second Convolution and Transistion block. RF achieved 20x20

    self.conv3 = nn.Sequential(
        nn.Conv2d(16, 32, 3), #Input=7x7x16 Kernel=3x3x16x32 Output=5x5x32 RF=22x22
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Conv2d(32, 32, 3), #Input=5x5x32 Kernel=3x3x32x32 Output=3x3x32 RF=24x24
        #nn.ReLU(),
        #nn.BatchNorm2d(32),
        nn.AdaptiveAvgPool2d(1), #Input=3x3x32 Kernel=3x3x32 Output=1x1x32
        nn.Conv2d(32, 10, 1) #Input=1x1x32 Kernel=1x1x32x10 Output=1x1x10
        ) #Final Convolution 


  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = x.view(-1, 10)
    return F.log_softmax(x)

In [23]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4           [-1, 10, 28, 28]             730
              ReLU-5           [-1, 10, 28, 28]               0
       BatchNorm2d-6           [-1, 10, 28, 28]              20
            Conv2d-7           [-1, 10, 28, 28]             910
              ReLU-8           [-1, 10, 28, 28]               0
       BatchNorm2d-9           [-1, 10, 28, 28]              20
        MaxPool2d-10           [-1, 10, 14, 14]               0
          Dropout-11           [-1, 10, 14, 14]               0
           Conv2d-12           [-1, 10, 14, 14]             910
             ReLU-13           [-1, 10, 14, 14]               0
      BatchNorm2d-14           [-1, 10,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [19]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [20]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [24]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 18):
    print(epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0654, Accuracy: 9800/10000 (98.00%)

2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0197, Accuracy: 9944/10000 (99.44%)

18


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0241, Accuracy: 9926/10000 (99.26%)

19


loss=0.009221390821039677 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 33.49it/s]



Test set: Average loss: 0.0227, Accuracy: 9933/10000 (99.33%)

